In [15]:
import argparse
from copy import copy
from hashlib import md5
import logging
import os
import re
import sys
import time

import yaxil
from yaxil.exceptions import RestApiError
from requests.exceptions import ConnectionError


In [16]:
if False or 1:
    print("lalala")

lalala


In [17]:
def download_file(sess,
                  uri,
                  digest,
                  collection,
                  out_dir='.',
                  overwrite=False,
                  **kwargs):
    basename = uri.split('files/')[-1]
    fname = os.path.join(out_dir, collection, basename)
    dirname = os.path.dirname(fname)

    if not os.path.exists(dirname):
        os.makedirs(dirname)
    if os.path.exists(fname):
        with open(fname, 'rb') as f:
            disk_digest = md5(f.read()).hexdigest()
        if disk_digest == digest:
            logger.debug('Digest matched - Skipping {}'.format(fname))
            return
        elif overwrite:
            # import pdb; pdb.set_trace()
            logger.info(
                'Digest failed - removing {} and trying again'.format(fname))
            os.remove(fname)
        else:
            raise RuntimeError(
                '{} exisited with incorrect digest '.format(fname) +
                'but cowardly moving on')

    try:
        _, result = yaxil._get(
            sess._auth,
            uri,
            yaxil.Format.JSON,  # Format is ignored for _file_ downloads
            autobox=False)
    except RestApiError as err:
        # Empty responses are acceptable for some scripts and onset files
        if 'response is empty' in str(err):
            result = bytes('', 'utf8')
        else:
            raise

    with open(fname, 'wb') as f:
        print('Writing ' + fname)
        f.write(result)

    with open(fname, 'rb') as f:
        disk_digest = md5(f.read()).hexdigest()
    if disk_digest != digest:
        retries = kwargs.get('retries', 0)
        if overwrite:
            os.remove(fname)
        if retries >= MAX_RETRIES:
            raise RuntimeError(
                'Digest failed - ' +
                '{} may need to be re-downloaded.'.format(fname))
        else:
            retries += 1
            download_file(sess,
                          uri,
                          digest,
                          collection,
                          out_dir,
                          overwrite=True,
                          retries=retries)


def ignore_file(uri, ignores_list):
    ignore = False
    for ignore_pat in ignores_list:
        if re.search(ignore_pat, uri):
            ignore = True
    return ignore

In [20]:
parser = argparse.ArgumentParser(description='Download from Remote XNAT')
parser.add_argument('--project', '-p', type=str)
parser.add_argument('--collections', '-c', type=str, nargs='+')
parser.add_argument('--ignore-list', type=str, nargs='+', default=['OTHER_FILES'])
parser.add_argument('--subjects', '-s', type=str, nargs='+', default=[], help='Explicit list of subjects')
parser.add_argument('--sessions', type=list, default=[], help='Explicit list of sessions')
parser.add_argument('--scantype', '-t', type=str, nargs='+')
parser.add_argument('--subcollection', '-C', type=str, nargs='+')

os.chdir('/net/holynfs01/srv/export/ncf_hcp/share_root/data/intradb/')
         
args = parser.parse_args(['-p', 'CCF_HCD_STG',  '-s', 'HCD1664358', '-c', 'Structural_preproc'])
opts = vars(args)
locals().update(opts)
opts

{'project': 'CCF_HCD_STG',
 'collections': ['Structural_preproc'],
 'ignore_list': ['OTHER_FILES'],
 'subjects': ['HCD1664358'],
 'sessions': [],
 'scantype': None,
 'subcollection': None}

In [21]:
if not ignore_list:
    ignore_list = list()
if not subjects:
    subjects = list()
if not sessions:
    sessions = list()
always_checksum=False

if(project == 'CCF_HCD_ITK'):
    print('Project is intake: testing')
project is 'CCF_HCD_ITK'

False

In [22]:
1 is not 2

True

In [23]:
auth = yaxil.auth('intradb')  # Requires setup and description
start_time = time.time()
subject_labels = subjects
logger = logging.getLogger(__name__)
MAX_RETRIES = 3
with yaxil.session(auth) as sess:
    if not sessions:
        """Fetch a list of yaxil.Experiment's (or get all for a project)."""
        if len(subject_labels):
            experiments = []
            for label in subject_labels:
                try:
                    sub = list(sess.subjects(label=label, project=project))[0]
                    experiments.extend(sess.experiments(subject=sub))
                except Exception as err:
                    print('Error with subject {}'.format(label))
                    print(str(err))
        else:
            experiments = list(sess.experiments(project=project))
        logger.info('Found {} experiments'.format(len(experiments)))

In [24]:
experiments

[Experiment(uri='/data/experiments/HCPIntradb09_E00967', label='HCD1664358_V1_MR', id='HCPIntradb09_E00967', project='CCF_HCD_STG', subject_id='HCPIntradb08_S00099', subject_label='HCD1664358', archived_date='2020-03-16 18:28:18.778')]

In [26]:
print(collections)
print(ignore_list)
print(always_checksum)
exp_info = experiments[0]._asdict()
print(exp_info)

['Structural_preproc']
['OTHER_FILES']
False
OrderedDict([('uri', '/data/experiments/HCPIntradb09_E00967'), ('label', 'HCD1664358_V1_MR'), ('id', 'HCPIntradb09_E00967'), ('project', 'CCF_HCD_STG'), ('subject_id', 'HCPIntradb08_S00099'), ('subject_label', 'HCD1664358'), ('archived_date', '2020-03-16 18:28:18.778')])


In [27]:
with yaxil.session(auth) as sess:
    logger.info('Syncing experiment {}'.format(exp_info['label']))
    start_time = time.time()

    # auth = yaxil.XnatAuth(url='...', username='...', password='...')

    # url = 'data/experiments/HCPIntradb02_E08367/files'  # ?format=json'
    # exp_info = dict(
    #     project='CCF_HCD_STG',
    #     subject_label='HCD0021614',
    #     label='HCD0021614_V1_MR')  # experiment_label

    """Fetch a list of json resources with collection label and id."""

    resources_url_pat = ('data/projects/{project}/subjects/{subject_label}/'
                         'experiments/{label}/resources')
    base_url = resources_url_pat.format(**exp_info)
    _, response = yaxil._get(sess._auth, base_url, yaxil.Format.JSON)

In [28]:
response['ResultSet']['Result']

[{'file_count': '12',
  'xnat_abstractresource_id': '3056770',
  'cat_desc': ' ',
  'cat_id': 'HCPIntradb09_E00967',
  'format': 'MISC',
  'label': 'rfMRI_REST1_PA_unproc',
  'category': 'resources',
  'element_name': 'xnat:resourceCatalog',
  'file_size': '563134980',
  'content': 'MISC',
  'tags': ''},
 {'file_count': '28',
  'xnat_abstractresource_id': '3056771',
  'cat_desc': ' ',
  'cat_id': 'HCPIntradb09_E00967',
  'format': 'MISC',
  'label': 'Diffusion_unproc',
  'category': 'resources',
  'element_name': 'xnat:resourceCatalog',
  'file_size': '940650173',
  'content': 'MISC',
  'tags': ''},
 {'file_count': '20',
  'xnat_abstractresource_id': '3056764',
  'cat_desc': ' ',
  'cat_id': 'HCPIntradb09_E00967',
  'format': 'MISC',
  'label': 'tfMRI_GUESSING_AP_unproc',
  'category': 'resources',
  'element_name': 'xnat:resourceCatalog',
  'file_size': '327233329',
  'content': 'MISC',
  'tags': ''},
 {'file_count': '27',
  'xnat_abstractresource_id': '3056759',
  'cat_desc': ' ',
  

In [29]:
collections

['Structural_preproc']

In [31]:
scantype=None
if collections:
    if scantype:
        resources = [
            result for result in response['ResultSet']['Result']
            if result['series_description'] in collections and result['type'] in scantype
        ]
    else :
        resources = [
            result for result in response['ResultSet']['Result']
            if result['series_description'] in collections
        ]
resources

KeyError: 'series_description'

In [55]:
resource_info = resources[0]
resource_info

{'xsiType': 'xnat:mrScanData',
 'xnat_imagescandata_id': '913327',
 'note': '',
 'series_description': 'T1w_setter',
 'ID': '107',
 'type': 'T1w_setter',
 'URI': '/data/experiments/HCPIntradb19_E00019/scans/107',
 'quality': 'usable'}

In [56]:
with yaxil.session(auth) as sess:
    resource_info = resources[0]
    resource_url_pat = (resource_info['URI'] + '/files')    

base_url = resource_url_pat
_, response = yaxil._get(sess._auth, base_url, yaxil.Format.JSON)
filelist = response['ResultSet']['Result']

In [57]:
response['ResultSet']['Result']

[{'file_content': '',
  'Size': '38861',
  'file_tags': '',
  'cat_ID': '2740658',
  'digest': '5f3182e0f5243871afc59b1ff7c032bf',
  'collection': 'NIFTI',
  'URI': '/data/experiments/HCPIntradb19_E00019/scans/107/resources/2740658/files/HCD0743955_V1_MR_T1w_setter.json',
  'file_format': '',
  'Name': 'HCD0743955_V1_MR_T1w_setter.json'},
 {'file_content': '',
  'Size': '4925175',
  'file_tags': '',
  'cat_ID': '2740658',
  'digest': '9a4abd4e73e05a4a75d79c52358e58ff',
  'collection': 'NIFTI',
  'URI': '/data/experiments/HCPIntradb19_E00019/scans/107/resources/2740658/files/HCD0743955_V1_MR_T1w_setter.nii.gz',
  'file_format': '',
  'Name': 'HCD0743955_V1_MR_T1w_setter.nii.gz'}]

In [58]:
print(base_url)
print(subcollection)
subcollection='NIFTI'

/data/experiments/HCPIntradb19_E00019/scans/107/files
['NIFTI']


In [59]:
filelist = [
    file for file in response['ResultSet']['Result'] 
    if file['collection'] in subcollection
]

In [60]:
filelist

[{'file_content': '',
  'Size': '38861',
  'file_tags': '',
  'cat_ID': '2740658',
  'digest': '5f3182e0f5243871afc59b1ff7c032bf',
  'collection': 'NIFTI',
  'URI': '/data/experiments/HCPIntradb19_E00019/scans/107/resources/2740658/files/HCD0743955_V1_MR_T1w_setter.json',
  'file_format': '',
  'Name': 'HCD0743955_V1_MR_T1w_setter.json'},
 {'file_content': '',
  'Size': '4925175',
  'file_tags': '',
  'cat_ID': '2740658',
  'digest': '9a4abd4e73e05a4a75d79c52358e58ff',
  'collection': 'NIFTI',
  'URI': '/data/experiments/HCPIntradb19_E00019/scans/107/resources/2740658/files/HCD0743955_V1_MR_T1w_setter.nii.gz',
  'file_format': '',
  'Name': 'HCD0743955_V1_MR_T1w_setter.nii.gz'}]

In [61]:
print(exp_info)
#filelist[0]['URI']
print(exp_info['label'])
os.getcwd()

OrderedDict([('uri', '/data/experiments/HCPIntradb19_E00019'), ('label', 'HCD0743955_V1_MR'), ('id', 'HCPIntradb19_E00019'), ('project', 'CCF_HCD_STG'), ('subject_id', 'HCPIntradb19_S00003'), ('subject_label', 'HCD0743955'), ('archived_date', '2019-10-09 11:24:05.575')])
HCD0743955_V1_MR


'/net/holynfs01/srv/export/ncf_hcp/share_root/data/intradb'

In [62]:
filelist[0]

{'file_content': '',
 'Size': '38861',
 'file_tags': '',
 'cat_ID': '2740658',
 'digest': '5f3182e0f5243871afc59b1ff7c032bf',
 'collection': 'NIFTI',
 'URI': '/data/experiments/HCPIntradb19_E00019/scans/107/resources/2740658/files/HCD0743955_V1_MR_T1w_setter.json',
 'file_format': '',
 'Name': 'HCD0743955_V1_MR_T1w_setter.json'}

In [63]:
resource_info_dir_index = 'series_description'
with yaxil.session(auth) as sess:
    if not len(filelist):
        raise ValueError('No files could be read from {} in json response: {}'.format(base_url, response))
    logger.info('Syncing {} file (resources) from {}'.format(len(filelist), base_url))

    start_time = time.time()
    for fileinfo in filelist[0:3]:
        try:
            # Rename URI (python variable case)
            fileinfo['uri'] = fileinfo.pop('URI')
            if ignore_file(fileinfo['uri'], ignore_list):
                logger.debug('Ignoring {}'.format(fileinfo['uri']))
                continue
            if project == 'CCF_HCD_ITK':
                out_dir = os.path.join(exp_info['label'], resource_info[resource_info_dir_index])
            else:
                out_dir = exp_info['label']
            download_file(sess, out_dir=out_dir, **fileinfo)
        except RuntimeError:
            logger.info('Digest failed on {}'.format(fileinfo['uri']))
        except RestApiError as err:
            logger.info('Download Error on {}: {}'.format(
                fileinfo['uri'], err))

Writing HCD0743955_V1_MR/NIFTI/HCD0743955_V1_MR_T1w_setter.json
Writing HCD0743955_V1_MR/NIFTI/HCD0743955_V1_MR_T1w_setter.nii.gz


In [ ]:
 

        # Filter only wanted collections or return all
        if collections:
            resources = [
                result for result in response['ResultSet']['Result']
                if result['label'] in collections
            ]
        else:
            resources = response['ResultSet']['Result']

        if not len(resources):
            msg = 'Found no resources '
            if collections:
                msg += 'matching collections {}'.format(collections)
            msg += 'for {}'.format(resources_url_pat)
            logger.warning(msg)

    except ValueError:
        logger.error('Unrecoverable error in {}'.format(exp_info['label']))
        resources = []

In [ ]:
    resource_errors = []

    for resource in resources:
        try:
            fetch_resource(sess, exp_info, resource, always_checksum=always_checksum, ignore_list=ignore_list)
        except ValueError as err:
            if 'No JSON object could be decoded' in str(err):
                logger.error(err)
                continue
            else:
                raise
        except ConnectionError as err:
            logger.error(err)
            resource_errors.append('ConnectionError: {}'.format(
                exp_info['label']))
            continue

    with open('errors.log', 'a') as f:
        f.writelines(resource_errors)
    elapsed_time = time.time() - start_time
    logger.info('Finished experiment {} in {}'.format(
        exp_info['label'], time.strftime("%H:%M:%S",
                                         time.gmtime(elapsed_time))))

In [ ]:
#!/usr/bin/env python
# Download Resources for a project

import argparse
from copy import copy
from hashlib import md5
import logging
import os
import re
import sys
import time

import yaxil
from yaxil.exceptions import RestApiError
from requests.exceptions import ConnectionError

logging.basicConfig(level=logging.DEBUG)a

logger = logging.getLogger(__name__)
MAX_RETRIES = 3


def main(project, collections, ignore_list=None, subjects=None, sessions=None, no_checksum=False):
    if not ignore_list:
        ignore_list = list()
    if not subjects:
        subjects = list()
    if not sessions:
        sessions = list()
    if no_checksum:
        always_checksum=False
    else:
        always_checksum=True
    auth = yaxil.auth('intradb')  # Requires setup and description
    start_time = time.time()
    with yaxil.session(auth) as sess:
        if not sessions:
            experiments = fetch_experiments(sess, project, subjects)

        for exp_info in [e._asdict() for e in experiments]:
            try:
                fetch_experiment(sess, collections, exp_info, ignore_list, always_checksum)
            except Exception as err:
                logger.error('Error with subject {}'.format(exp_info['label']))
                continue
    elapsed_time = time.time() - start_time
    logger.info('Finished {} experiments in {}'.format(
        len(experiments), time.strftime("%H:%M:%S",
                                        time.gmtime(elapsed_time))))


def fetch_experiments(sess, project, subject_labels):
    """Fetch a list of yaxil.Experiment's (or get all for a project)."""
    logger.info('Fetching list of experiments')
    if len(subject_labels):
        experiments = []
        for label in subject_labels:
            try:
                sub = list(sess.subjects(label=label, project=project))[0]
                experiments.extend(sess.experiments(subject=sub))
            except Exception as err:
                print('Error with subject {}'.format(label))
                print(str(err))
    else:
        experiments = list(sess.experiments(project=project))
    logger.info('Found {} experiments'.format(len(experiments)))
    return experiments

def fetch_experiment(sess, collections, exp_info, ignore_list, always_checksum):
    logger.info('Syncing experiment {}'.format(exp_info['label']))
    start_time = time.time()

    # auth = yaxil.XnatAuth(url='...', username='...', password='...')

    # url = 'data/experiments/HCPIntradb02_E08367/files'  # ?format=json'
    # exp_info = dict(
    #     project='CCF_HCD_STG',
    #     subject_label='HCD0021614',
    #     label='HCD0021614_V1_MR')  # experiment_label

    try:
        resources = fetch_resources(sess, exp_info, collections)
    except ValueError:
        logger.error('Unrecoverable error in {}'.format(exp_info['label']))
        resources = []

    resource_errors = []

    for resource in resources:
        try:
            fetch_resource(sess, exp_info, resource, always_checksum=always_checksum, ignore_list=ignore_list)
        except ValueError as err:
            if 'No JSON object could be decoded' in str(err):
                logger.error(err)
                continue
            else:
                raise
        except ConnectionError as err:
            logger.error(err)
            resource_errors.append('ConnectionError: {}'.format(
                exp_info['label']))
            continue

    with open('errors.log', 'a') as f:
        f.writelines(resource_errors)
    elapsed_time = time.time() - start_time
    logger.info('Finished experiment {} in {}'.format(
        exp_info['label'], time.strftime("%H:%M:%S",
                                         time.gmtime(elapsed_time))))

    # import pdb
    # pdb.set_trace()
    # url_pat = ('data/projects/{project}/subjects/{subject_label}/experiments/'
    #            '{label}/files')
    #
    # base_url = url_pat.format(**exp_info)
    # logger.info('Syncing {}'.format(base_url))

    # import pdb
    # pdb.set_trace()
    # # sess.get(base_url, yaxil.Format.JSON)
    # _, response = yaxil._get(sess._auth, base_url, yaxil.Format.JSON)
    #
    # for collection in collections:
    #     import pdb
    #     pdb.set_trace()
    #     logger.info('Downloading {}'.format(collection))
    #     filelist = [
    #         r for r in response['ResultSet']['Result']
    #         if r['collection'] == collection
    #     ]


def fetch_resources(sess, exp_info, collections=None):
    """Fetch a list of json resources with collection label and id."""

    resources_url_pat = ('data/projects/{project}/subjects/{subject_label}/'
                         'experiments/{label}/resources')
    base_url = resources_url_pat.format(**exp_info)
    _, response = yaxil._get(sess._auth, base_url, yaxil.Format.JSON)

    # Filter only wanted collections or return all
    if collections:
        resources = [
            result for result in response['ResultSet']['Result']
            if result['label'] in collections
        ]
    else:
        resources = response['ResultSet']['Result']

    if not len(resources):
        msg = 'Found no resources '
        if collections:
            msg += 'matching collections {}'.format(collections)
        msg += 'for {}'.format(resources_url_pat)
        logger.warning(msg)

    return resources


def fetch_resource(sess, exp_info, resource_info, always_checksum=False, ignore_list=None):
    if not ignore_list:
        ignore_list = list()
    # Use a cookie to mark a resource as complete
    success_cookie = os.path.join(exp_info['label'], resource_info['label'],
                                  'SUCCESS')
    if os.path.exists(success_cookie) and not always_checksum:
        return

    resource_url_pat = (
        'data/projects/{project}/subjects/{subject_label}/'
        'experiments/{label}/resources/{xnat_abstractresource_id}/files')
    url_info = copy(exp_info)  # Combine resource and experiment
    url_info['xnat_abstractresource_id'] = resource_info[
        'xnat_abstractresource_id']

    base_url = resource_url_pat.format(**url_info)
    _, response = yaxil._get(sess._auth, base_url, yaxil.Format.JSON)

    filelist = response['ResultSet']['Result']
    if not len(filelist):
        raise ValueError('No files could be read from {} in json response: {}'.format(base_url, response))
    logger.info('Syncing {} file (resources) from {}'.format(len(filelist), base_url))

    start_time = time.time()
    for fileinfo in filelist:
        try:
            # Rename URI (python variable case)
            fileinfo['uri'] = fileinfo.pop('URI')
            if ignore_file(fileinfo['uri'], ignore_list):
                logger.debug('Ignoring {}'.format(fileinfo['uri']))
                continue
            download_file(sess, out_dir=exp_info['label'], **fileinfo)
        except RuntimeError:
            logger.info('Digest failed on {}'.format(fileinfo['uri']))
        except RestApiError as err:
            logger.info('Download Error on {}: {}'.format(
                fileinfo['uri'], err))

    elapsed_time = time.time() - start_time
    logger.info('Finished in {}'.format(
        time.strftime("%H:%M:%S", time.gmtime(elapsed_time))))

    # if we got here, mark this collection as completed ("touch" cookie)
    open(success_cookie, 'w').close()


def download_file(sess,
                  uri,
                  digest,
                  collection,
                  out_dir='.',
                  overwrite=False,
                  **kwargs):
    basename = uri.split('files/')[-1]
    fname = os.path.join(out_dir, collection, basename)
    dirname = os.path.dirname(fname)

    if not os.path.exists(dirname):
        os.makedirs(dirname)
    if os.path.exists(fname):
        with open(fname, 'rb') as f:
            disk_digest = md5(f.read()).hexdigest()
        if disk_digest == digest:
            logger.debug('Digest matched - Skipping {}'.format(fname))
            return
        elif overwrite:
            # import pdb; pdb.set_trace()
            logger.info(
                'Digest failed - removing {} and trying again'.format(fname))
            os.remove(fname)
        else:
            raise RuntimeError(
                '{} exisited with incorrect digest '.format(fname) +
                'but cowardly moving on')

    try:
        _, result = yaxil._get(
            sess._auth,
            uri,
            yaxil.Format.JSON,  # Format is ignored for _file_ downloads
            autobox=False)
    except RestApiError as err:
        # Empty responses are acceptable for some scripts and onset files
        if 'response is empty' in str(err):
            result = bytes('', 'utf8')
        else:
            raise

    with open(fname, 'wb') as f:
        f.write(result)

    with open(fname, 'rb') as f:
        disk_digest = md5(f.read()).hexdigest()
    if disk_digest != digest:
        retries = kwargs.get('retries', 0)
        if overwrite:
            os.remove(fname)
        if retries >= MAX_RETRIES:
            raise RuntimeError(
                'Digest failed - ' +
                '{} may need to be re-downloaded.'.format(fname))
        else:
            retries += 1
            download_file(sess,
                          uri,
                          digest,
                          collection,
                          out_dir,
                          overwrite=True,
                          retries=retries)


def ignore_file(uri, ignores_list):
    ignore = False
    for ignore_pat in ignores_list:
        if re.search(ignore_pat, uri):
            ignore = True
    return ignore


def parse_args():
    parser = argparse.ArgumentParser(description='Download from Remote XNAT')
    parser.add_argument('--project', '-p', type=str)
    parser.add_argument('--collections', '-c', type=str, nargs='+')
    parser.add_argument('--ignore-list', type=str, nargs='+', default=['OTHER_FILES'])
    parser.add_argument('--subjects', '-s', type=str, nargs='+', default=[], help='Explicit list of subjects')
    parser.add_argument('--sessions', type=list, default=[], help='Explicit list of sessions')
    parser.add_argument('--no-checksum', action='store_true')

    return parser.parse_args()

In [ ]:
parser = argparse.ArgumentParser(description='Download from Remote XNAT')
parser.add_argument('--project', '-p', type=str)
parser.add_argument('--collections', '-c', type=str, nargs='+')
parser.add_argument('--ignore-list', type=str, nargs='+', default=['OTHER_FILES'])
parser.add_argument('--subjects', '-s', type=str, nargs='+', default=[], help='Explicit list of subjects')
parser.add_argument('--sessions', type=list, default=[], help='Explicit list of sessions')

os.chdir('/net/holynfs01/srv/export/ncf_hcp/share_root/data/intradb/')
         
args = parser.parse_args(['-p', 'CCF_HCD_ITK',  '-s', 'HCD0737657', 'HCD0551239'])
opts = vars(args)
opts

In [ ]:
locals().update(opts)
opts

In [ ]:
if not ignore_list:
    ignore_list = list()
if not subjects:
    subjects = list()
if not sessions:
    sessions = list()
if no_checksum:
    always_checksum=False
else:
    always_checksum=True
auth = yaxil.auth('intradb')  # Requires setup and description
start_time = time.time()
with yaxil.session(auth) as sess:
    if not sessions:
        experiments = fetch_experiments(sess, project, subjects)

    for exp_info in [e._asdict() for e in experiments]:
        try:
            fetch_experiment(sess, collections, exp_info, ignore_list, always_checksum)
        except Exception as err:
            logger.error('Error with subject {}'.format(exp_info['label']))
            continue
elapsed_time = time.time() - start_time
logger.info('Finished {} experiments in {}'.format(
    len(experiments), time.strftime("%H:%M:%S",
                                    time.gmtime(elapsed_time))))
